<a href="https://colab.research.google.com/github/Israel-Mafabi-Emmanuel/Israel-Mafabi-Emmanuel/blob/main/mafabi_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Private Image Generation 101

#### From Zero to Local Flask API on Google Colab

#### Step 1: Environment Setup
First we need to install the libraries. I've included ```accelerate``` and ```diffusers```, which are gold standards for this.







In [1]:
# 1. Install necessary libraries
# 'diffusers'    -> a library that handles the AI models
# 'transformers' -> a library that helps with text processing
# 'accelerate'   -> a library that optimizes the model for the GPU
!pip install -U diffusers transformers accelerate bitsandbytes huggingface_hub flask

In [2]:
from huggingface_hub import login
print("🔑 Please paste your Hugging Face token below (make sure you accepted the terms on the model page!):")
login()

🔑 Please paste your Hugging Face token below (make sure you accepted the terms on the model page!):


#### Step 2: Loading the Model
We'll use **Stable Diffusion v1.5**. It's open-source and very efficient.
*Note: "We use float16 precision to save memory so the free Colab GPU doesn't get overwhelmed."*

In [3]:
import torch
import gc

def clear_memory():
    # Delete the pipe if it exists
    global pipe
    if 'pipe' in globals():
        del pipe

    # Garbage collection
    gc.collect()

    # Clear CUDA cache
    torch.cuda.empty_cache()
    print("🧹 Memory cleared! The vessel is empty and ready.")

clear_memory()

🧹 Memory cleared! The vessel is empty and ready.


In [3]:
import torch
from diffusers import StableDiffusion3Pipeline
from transformers import T5EncoderModel, BitsAndBytesConfig

# 1. We load the 'Heavy Brain' (T5) in a compressed 8-bit format
# This is the secret for 12GB RAM Colab sessions!
t5_8bit_config = BitsAndBytesConfig(load_in_8bit=True)

print("⏳ Loading the 'Lightweight Brain' (T5)...")
text_encoder_3 = T5EncoderModel.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    subfolder="text_encoder_3",
    quantization_config=t5_8bit_config,
    torch_dtype=torch.float16
)

# 2. Now we load the rest of the pipeline using that lightweight brain
print("🎨 Loading the rest of SD 3.5 Medium...")
pipe = StableDiffusion3Pipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-medium",
    text_encoder_3=text_encoder_3,
    torch_dtype=torch.float16,
)

# 3. Apply extreme optimizations
pipe.enable_model_cpu_offload() # Moves parts to GPU only when needed
pipe.enable_attention_slicing() # Processes image in chunks to save VRAM

print("✅ SD 3.5 Medium is ready and 'System RAM' safe!")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.


⏳ Loading the 'Lightweight Brain' (T5)...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/219 [00:00<?, ?it/s]

🎨 Loading the rest of SD 3.5 Medium...


Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/517 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

The module 'T5EncoderModel' has been loaded in `bitsandbytes` 8bit and moving it to cpu via `.to()` is not supported. Module is still on cuda:0.


✅ SD 3.5 Medium is ready and 'System RAM' safe!


#### Step 3: Creating the Generation Function
Before we put it in a Flask server, let's make sure it works with a simple Python function.

In [6]:
import re

# A function for cleaning the prompt ~ stripping out extra spaces, remove newline characters, and ensuring the text is formatted perfectly for the pipeline
def clean_prompt(prompt:str):
  # 1. Remove new lines and tabs (they occur when copy-pasting)
  text = prompt.replace("\n", " ").replace("\t", " ")

  # 2. Replace multiple spaces and replace with a single space
  text = re.sub(' +', ' ', prompt)

  # 3. Strip leading and trailing whitespace
  text = prompt.strip()

  return text

def generate_image(prompt:str, negative_prompt:str="", filename="output.png"):
  # Sanitize the inputs...
  refined_prompt = clean_prompt(prompt)
  refined_negative = clean_prompt(negative_prompt)

  # The 'prompt' is what you want the AI to draw
  # Adding a 'negative_prompt' helps the AI avoid common mistakes
  # The standard 'anti-abomination' string:
  default_negative = "extra fingers, deformed hands, distorted face, extra limbs" # I have not mentioned ~ quality wise, as some prefer the natural, normal (not studio-based) shots.
  full_negative = f"{refined_negative}, {default_negative}"

  print(f"🚀 Processing your vision: {refined_prompt[:50]}...")
  print(f"⚠️ Negative prompt: {full_negative}")

  image = pipe(
    prompt=refined_prompt,
    negative_prompt=full_negative,
    num_inference_steps=30, # 30-50 is the sweet spot for quality vs speed
    guidance_scale=4.5,
    height=896,
    width=896
  ).images[0]
  image.save(filename)
  print(f"✨ Generation Complete! Saved as {filename}")
  return filename

# Test run
generate_image("Low-angle shot of a stunning 25-year-old African woman, having black braids and wearing a black and green kitenge dress, standing in a dimly lit wooden shed, Her hands behind her and are not seen. She has closed eyes. Curvy build, small waist, wide hips. Natural dark skin texture with visible pores and blemishes. Soft, cool diffused lighting.")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', cool diffused lighting.']
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', cool diffused lighting.']


🚀 Processing your vision: Low-angle shot of a stunning 25-year-old African w...
⚠️ Negative prompt: , extra fingers, deformed hands, distorted face, extra limbs


  0%|          | 0/30 [00:00<?, ?it/s]

The module 'T5EncoderModel' has been loaded in `bitsandbytes` 8bit and moving it to cpu via `.to()` is not supported. Module is still on cuda:0.


✨ Generation Complete! Saved as output.png


'output.png'

#### Step 4: The Flask Server Route
Now, let's wrap this in a Flask application. This allows other "services" or "front-ends" to talk to our AI model.

In [ ]:
from flask import Flask, request, jsonify, send_file
import io

app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def api_generate():
  data = request.json
  prompt = data.get('prompt', "A beautiful sunset")

  # Generate the image
  image = pipe(prompt).images[0]

  # Save the image to a buffer to send over the web
  img_io = io.BytesIO()
  image.save(img_io, 'PNG')
  img_io.seek(0)

  return send_file(img_io, mimetype='image/png')

# Note: In Colab, we can't just 'run' Flask normally and see it.
# We will use a trick to run it in the background or use a tunnel later.

#### Step 5: How to Run it on Colab
Using multithreading ~ Running the server in the background but still you can use the notebook.

In [ ]:
import threading

# Function to run the app
def run_app():
  app.run(port=5000)

# Start Flask in a separate thread so it doesn't freeze the notebook
threading.Thread(target=run_app).start()